In [1]:
import tensorflow as tf

In [2]:
corpus=["food is very good","food is tasty",'awesome food']

In [4]:
tk=tf.keras.preprocessing.text.Tokenizer()

In [6]:
tk.fit_on_texts(corpus)

In [7]:
tk.word_index

{'food': 1, 'is': 2, 'very': 3, 'good': 4, 'tasty': 5, 'awesome': 6}

In [8]:
seqs=tk.texts_to_sequences(corpus)

In [9]:
seqs

[[1, 2, 3, 4], [1, 2, 5], [6, 1]]

In [10]:
#padding is required 

In [14]:
pad_seqs=tf.keras.preprocessing.sequence.pad_sequences(seqs,padding='post',maxlen=4)
pad_seqs

array([[1, 2, 3, 4],
       [1, 2, 5, 0],
       [6, 1, 0, 0]])

In [16]:
one_hot=tf.keras.utils.to_categorical(pad_seqs)

In [17]:
one_hot

array([[[0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0.]],

       [[0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.]]], dtype=float32)

In [18]:
import pandas as pd

In [21]:
df=pd.read_csv("Restaurant_Reviews.tsv",delimiter="\t")

In [34]:
corpus=df.Review

In [35]:
y=df.Liked

In [56]:
import nltk
sw=nltk.corpus.stopwords.words("english")
sw.remove('not')
sw.remove('no')
import re
def clean(doc):
    doc=doc.lower()
    doc=re.sub("[^a-z ]", "",doc)
    string=""
    for i in doc.split():
        if i not in sw:
            string+=i+" "      
    return string.strip()

In [57]:
clean_corpus=list(map(clean,corpus))

In [59]:
tk=tf.keras.preprocessing.text.Tokenizer()

In [65]:
tk.fit_on_texts(clean_corpus)

In [84]:
tk.word_index
seq=tk.texts_to_sequences(clean_corpus)
pad_seq=tf.keras.preprocessing.sequence.pad_sequences(seq,padding="post")
one_hot_pad_seq=tf.keras.utils.to_categorical(pad_seq)

In [86]:
one_hot_pad_seq.shape

(1000, 20, 1939)

In [93]:
model=tf.keras.Sequential([
tf.keras.layers.SimpleRNN(10,input_shape=(20,1939)),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(50,activation='relu'),
tf.keras.layers.Dense(2,activation='softmax')])
model.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.sparse_categorical_crossentropy,metrics=['accuracy'])
model.fit(one_hot_pad_seq,y,epochs=5)

Epoch 1/5
32/32 [==============================] - 2s 14ms/step - loss: 0.6935 - accuracy: 0.4960
Epoch 2/5
32/32 [==============================] - 0s 13ms/step - loss: 0.6603 - accuracy: 0.7030
Epoch 3/5
32/32 [==============================] - 0s 13ms/step - loss: 0.4689 - accuracy: 0.8360
Epoch 4/5
32/32 [==============================] - 0s 13ms/step - loss: 0.1742 - accuracy: 0.9550
Epoch 5/5
32/32 [==============================] - 0s 13ms/step - loss: 0.0495 - accuracy: 0.9910


In [104]:
sample="food is not good"
clean_sample_corpus=clean(sample)

In [98]:
clean_sample_corpus

'food not good'

In [108]:
seq_clean_sample_corpus=tk.texts_to_sequences([clean_sample_corpus])

In [109]:
seq_clean_sample_corpus

[[1, 2, 4]]

In [118]:
pad_seq_seq_clean_sample_corpus=tf.keras.preprocessing.sequence.pad_sequences(seq_clean_sample_corpus,padding='post',maxlen=20)

In [121]:
sample_matrix=tf.keras.utils.to_categorical(pad_seq_seq_clean_sample_corpus,num_classes=1939)

In [122]:
model.predict(sample_matrix)

1/1 [==============================] - 0s 317ms/step


array([[0.9642228 , 0.03577723]], dtype=float32)